In [2]:
import pandas as pd
import numpy as np
import os
import simplejson


In [3]:
def load_file(path):
    file_path = path
    df = pd.read_csv(file_path)
    #Automatic fixes
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df


In [4]:
def file_params(df):
    params = {}
    #size
    params['size'] = {}
    params['size']['rows'] = int(df.shape[0])
    params['size']['cols'] = int(df.shape[1])

    #missing values
    params['missing'] = {}
    ##rows
    params['missing']['rows'] = int(df[df.isna().any(axis=1)].shape[0])
    params['missing']['rowsPercent'] = float(round(params['missing']['rows'] / params['size']['rows'], 7) * 100)
    ##cells
    params['missing']['cells'] = int(df.isna().sum().sum())
    params['missing']['cellsPercent'] = float(round(params['missing']['cells'] / (params['size']['rows'] * params['size']['cols']), 7) * 100)
    ##columns
    missingColumn = df.isna().sum()
    params['missing']['cols'] = missingColumn[missingColumn != 0].to_dict()
    ###percent of total values in colums
    nanByColumnPercent = round(df.isna().sum() / df.count().sum() * 100, 4)
    params['missing']['colsPercent'] = nanByColumnPercent[nanByColumnPercent !=0].to_dict()
    ###percent ot total missing values
    nanColumnContributionPercent = round(df.isna().sum() / df.isna().sum().sum() * 100, 2)
    params['missing']['colsPercentContribution'] = nanColumnContributionPercent[nanColumnContributionPercent !=0].to_dict()

    #names
    params['names'] = {}
    params['names']['cols'] = list(df.columns.values)
    params['names']['colsReverse'] = list(df.columns.values)
    params['names']['colsReverse'].reverse()

    #describe
    params['describe'] = df.describe().to_dict()

    return params

In [19]:
path =  ''
df_si = '123'
df2_si = '456'

df_sub1 = load_file('/Users/brandon/Desktop/test.csv')
df_sub1['storage_id'] = df_si
df_sub2 = load_file('/Users/brandon/Desktop/train.csv')
df_sub2['storage_id'] = df2_si

df = pd.concat([df_sub1, df_sub2])

id = df['storage_id']

df.drop(['storage_id'], axis=1, inplace=True)




In [20]:
valid_data_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']

valid = df.select_dtypes(include=valid_data_types)

invalid = df.drop(columns=valid.columns)

In [25]:
result = []

for column in invalid.columns:
    col = invalid[column]

    total_count = col.shape[0]
    nan_count = col.isna().sum()

    col = col.dropna()
    not_nan_count = col.shape[0]

    numeric = ~pd.to_numeric(col, errors='coerce').isna()
    numeric_count = numeric.sum()

    if (numeric_count / total_count) > 0.6:
        invalid[column] = pd.to_numeric(col, errors='coerce')
        #proposed_transform
        result.append({
            'col': col,
            'method': 'mixed_to_numeric',
            'nan_row_index': list(col[col.isna() == True].index),
            'non_numeric_row_index': list(numeric[numeric == False].index)
        })
    else:
        result.append({
            'col': col,
            'method': 'one_hot_encode',
            'unique_values': list(col.unique()),
            'nan_row_index': list(col[col.isna() == True].index),
            'keep_column': True if len(list(col.unique())) <=20 else False #rule to decide if column should be dropped by default
        })


In [26]:
transforms

{'cat1': {'type': 'one_hot_encode',
  'unique_values': ['A', 'B', 'C'],
  'nan_row_index': [],
  'keep_column': True},
 'cat2': {'type': 'one_hot_encode',
  'unique_values': ['one', 'two'],
  'nan_row_index': [],
  'keep_column': True},
 'num_cont': {'type': 'mixed_to_numeric',
  'nan_row_index': [],
  'non_numeric_row_index': [0, 0]}}